<a href="https://colab.research.google.com/github/visist16/TDS/blob/main/Copy_of_BBC_weather_scrape.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# A tutorial to scrape the web. 

This example scrapes the BBC weather website for any specific city, and collects weather forecast for the next 14 days and saves it as a csv file.

*Web scraping might not be legal always. It is a good idea to check the terms of the website you plan to scrape before proceeding. Also, if your code requests a url from a server multiple times, it is a good practice to either cache your requests, or insert a timed delay between consecutive requests.*

In [ ]:
import json                   # to convert API to json format

from urllib.parse import urlencode

import requests               # to get the webpage
from bs4 import BeautifulSoup # to parse the webpage

import pandas as pd
import re                     # regular expression operators

from datetime import datetime

We now GET the webpage of interest, from the server

In [ ]:
required_city = "Mumbai"
location_url = 'https://locator-service.api.bbci.co.uk/locations?' + urlencode({
   'api_key': 'AGbFAKx58hyjQScCXIYrxuEwJh2W2cmv',
   's': required_city,
   'stack': 'aws',
   'locale': 'en',
   'filter': 'international',
   'place-types': 'settlement,airport,district',
   'order': 'importance',
   'a': 'true',
   'format': 'json'
})
location_url

'https://locator-service.api.bbci.co.uk/locations?api_key=AGbFAKx58hyjQScCXIYrxuEwJh2W2cmv&s=Mumbai&stack=aws&locale=en&filter=international&place-types=settlement%2Cairport%2Cdistrict&order=importance&a=true&format=json'

In [ ]:
result = requests.get(location_url).json()
result

{'response': {'results': {'results': [{'id': '1275339',
     'name': 'Mumbai',
     'container': 'India',
     'containerId': 1269750,
     'language': 'en',
     'timezone': 'Asia/Kolkata',
     'country': 'IN',
     'latitude': 19.07283,
     'longitude': 72.88261,
     'placeType': 'settlement'}],
   'totalResults': 1}}}

In [ ]:
# url      = 'https://www.bbc.com/weather/1275339' # url to BBC weather, corresponding to a specific city (Mumbai, in this example)
url      = 'https://www.bbc.com/weather/'+result['response']['results']['results'][0]['id']
response = requests.get(url)

Next, we initiate an instance of  BeautifulSoup.

In [ ]:
soup = BeautifulSoup(response.content,'html.parser') 

The information we want (daily high and low temp., and daily weather summary), are in specific blocks on the webpage. 
We need to find the block type, type of identifier, and the identifier name (all these can be figured out by right clicking
on the webpage and selecting 'Inspect' on the Chrome browser; similar modus operandi for other browsers)

In [ ]:
daily_high_values = soup.find_all('span', attrs={'class': 'wr-day-temperature__high-value'}) # block-type: span; identifier type: class; and class name: wr-day-temperature__high-value 
daily_high_values

[<span class="wr-day-temperature__high-value"><span class="wr-value--temperature "><span class="wr-value--temperature--c">33°</span><span class="wr-hide"> </span><span class="wr-value--temperature--f">92°</span></span></span>,
 <span class="wr-day-temperature__high-value"><span class="wr-value--temperature "><span class="wr-value--temperature--c">34°</span><span class="wr-hide"> </span><span class="wr-value--temperature--f">93°</span></span></span>,
 <span class="wr-day-temperature__high-value"><span class="wr-value--temperature "><span class="wr-value--temperature--c">34°</span><span class="wr-hide"> </span><span class="wr-value--temperature--f">93°</span></span></span>,
 <span class="wr-day-temperature__high-value"><span class="wr-value--temperature "><span class="wr-value--temperature--c">34°</span><span class="wr-hide"> </span><span class="wr-value--temperature--f">94°</span></span></span>,
 <span class="wr-day-temperature__high-value"><span class="wr-value--temperature "><span cla

In [ ]:
daily_low_values  = soup.find_all('span', attrs={'class': 'wr-day-temperature__low-value'})
daily_low_values

[<span class="wr-day-temperature__low-value"><span class="wr-value--temperature "><span class="wr-value--temperature--c">22°</span><span class="wr-hide"> </span><span class="wr-value--temperature--f">71°</span></span></span>,
 <span class="wr-day-temperature__low-value"><span class="wr-value--temperature "><span class="wr-value--temperature--c">22°</span><span class="wr-hide"> </span><span class="wr-value--temperature--f">71°</span></span></span>,
 <span class="wr-day-temperature__low-value"><span class="wr-value--temperature "><span class="wr-value--temperature--c">22°</span><span class="wr-hide"> </span><span class="wr-value--temperature--f">72°</span></span></span>,
 <span class="wr-day-temperature__low-value"><span class="wr-value--temperature "><span class="wr-value--temperature--c">22°</span><span class="wr-hide"> </span><span class="wr-value--temperature--f">71°</span></span></span>,
 <span class="wr-day-temperature__low-value"><span class="wr-value--temperature "><span class="w

In [ ]:
daily_summary = soup.find('div', attrs={'class': 'wr-day-summary'})
daily_summary

<div class="wr-day-summary"><div class="gel-wrap"><span class="">A clear sky and light winds</span><span class="wr-hide">Sunny intervals and a gentle breeze</span><span class="wr-hide">Sunny and a gentle breeze</span><span class="wr-hide">Sunny and a gentle breeze</span><span class="wr-hide">Sunny and a gentle breeze</span><span class="wr-hide">Sunny intervals and a gentle breeze</span><span class="wr-hide">Sunny intervals and a gentle breeze</span><span class="wr-hide">Sunny intervals and a gentle breeze</span><span class="wr-hide">Sunny intervals and a gentle breeze</span><span class="wr-hide">Sunny intervals and a gentle breeze</span><span class="wr-hide">Sunny intervals and a gentle breeze</span><span class="wr-hide">Sunny intervals and a gentle breeze</span><span class="wr-hide">Sunny intervals and a gentle breeze</span><span class="wr-hide">Sunny intervals and a gentle breeze</span></div></div>

In [ ]:
daily_summary.text

'A clear sky and light windsSunny intervals and a gentle breezeSunny and a gentle breezeSunny and a gentle breezeSunny and a gentle breezeSunny intervals and a gentle breezeSunny intervals and a gentle breezeSunny intervals and a gentle breezeSunny intervals and a gentle breezeSunny intervals and a gentle breezeSunny intervals and a gentle breezeSunny intervals and a gentle breezeSunny intervals and a gentle breezeSunny intervals and a gentle breeze'

General book keeping.

With the code snippet in the cell above, we get forecast data for 14 days, including today. We will now post process the data to first extract the required information/text and discard all the html wrapper code, then combine all variables into one common list, and finally convert it into a pandas data frame.

In [ ]:
daily_high_values[0].text.strip()

'33° 92°'

In [ ]:
daily_high_values[5].text.strip()

'34° 93°'

In [ ]:
daily_high_values[0].text.strip().split()[0]

'33°'

In [ ]:
daily_high_values_list = [daily_high_values[i].text.strip().split()[0] for i in range(len(daily_high_values))]
daily_high_values_list

['33°',
 '34°',
 '34°',
 '34°',
 '34°',
 '34°',
 '33°',
 '33°',
 '34°',
 '34°',
 '33°',
 '33°',
 '33°']

In [ ]:
daily_low_values_list = [daily_low_values[i].text.strip().split()[0] for i in range(len(daily_low_values))]
daily_low_values_list

['22°',
 '22°',
 '22°',
 '22°',
 '22°',
 '23°',
 '23°',
 '23°',
 '24°',
 '24°',
 '23°',
 '23°',
 '23°',
 '23°']

In [ ]:
daily_summary.text

'A clear sky and light windsSunny intervals and a gentle breezeSunny and a gentle breezeSunny and a gentle breezeSunny and a gentle breezeSunny intervals and a gentle breezeSunny intervals and a gentle breezeSunny intervals and a gentle breezeSunny intervals and a gentle breezeSunny intervals and a gentle breezeSunny intervals and a gentle breezeSunny intervals and a gentle breezeSunny intervals and a gentle breezeSunny intervals and a gentle breeze'

In [ ]:
daily_summary_list = re.findall('[a-zA-Z][^A-Z]*', daily_summary.text) #split the string on uppercase
daily_summary_list

['A clear sky and light winds',
 'Sunny intervals and a gentle breeze',
 'Sunny and a gentle breeze',
 'Sunny and a gentle breeze',
 'Sunny and a gentle breeze',
 'Sunny intervals and a gentle breeze',
 'Sunny intervals and a gentle breeze',
 'Sunny intervals and a gentle breeze',
 'Sunny intervals and a gentle breeze',
 'Sunny intervals and a gentle breeze',
 'Sunny intervals and a gentle breeze',
 'Sunny intervals and a gentle breeze',
 'Sunny intervals and a gentle breeze',
 'Sunny intervals and a gentle breeze']

In [ ]:
datelist = pd.date_range(datetime.today(), periods=len(daily_high_values)).tolist()
datelist

[Timestamp('2022-11-01 16:46:53.482769', freq='D'),
 Timestamp('2022-11-02 16:46:53.482769', freq='D'),
 Timestamp('2022-11-03 16:46:53.482769', freq='D'),
 Timestamp('2022-11-04 16:46:53.482769', freq='D'),
 Timestamp('2022-11-05 16:46:53.482769', freq='D'),
 Timestamp('2022-11-06 16:46:53.482769', freq='D'),
 Timestamp('2022-11-07 16:46:53.482769', freq='D'),
 Timestamp('2022-11-08 16:46:53.482769', freq='D'),
 Timestamp('2022-11-09 16:46:53.482769', freq='D'),
 Timestamp('2022-11-10 16:46:53.482769', freq='D'),
 Timestamp('2022-11-11 16:46:53.482769', freq='D'),
 Timestamp('2022-11-12 16:46:53.482769', freq='D'),
 Timestamp('2022-11-13 16:46:53.482769', freq='D')]

In [ ]:
datelist = [datelist[i].date().strftime('%y-%m-%d') for i in range(len(datelist))]
datelist

['22-11-01',
 '22-11-02',
 '22-11-03',
 '22-11-04',
 '22-11-05',
 '22-11-06',
 '22-11-07',
 '22-11-08',
 '22-11-09',
 '22-11-10',
 '22-11-11',
 '22-11-12',
 '22-11-13']

In [ ]:
zipped = zip(datelist, daily_high_values_list, daily_low_values_list, daily_summary_list)

In [ ]:
df = pd.DataFrame(list(zipped), columns=['Date', 'High','Low', 'Summary'])

In [ ]:
display(df)

,Date,High,Low,Summary
0,22-11-01,33°,22°,A clear sky and light winds
1,22-11-02,34°,22°,Sunny intervals and a gentle breeze
2,22-11-03,34°,22°,Sunny and a gentle breeze
3,22-11-04,34°,22°,Sunny and a gentle breeze
4,22-11-05,34°,22°,Sunny and a gentle breeze
5,22-11-06,34°,23°,Sunny intervals and a gentle breeze
6,22-11-07,33°,23°,Sunny intervals and a gentle breeze
7,22-11-08,33°,23°,Sunny intervals and a gentle breeze
8,22-11-09,34°,24°,Sunny intervals and a gentle breeze
9,22-11-10,34°,24°,Sunny intervals and a gentle breeze


In [ ]:
# remove the 'degree' character
df.High = df.High.replace('\°','',regex=True).astype(float)
df.Low  = df.Low.replace('\°','',regex=True).astype(float)

In [ ]:
display(df)

,Date,High,Low,Summary
0,22-11-01,33.0,22.0,A clear sky and light winds
1,22-11-02,34.0,22.0,Sunny intervals and a gentle breeze
2,22-11-03,34.0,22.0,Sunny and a gentle breeze
3,22-11-04,34.0,22.0,Sunny and a gentle breeze
4,22-11-05,34.0,22.0,Sunny and a gentle breeze
5,22-11-06,34.0,23.0,Sunny intervals and a gentle breeze
6,22-11-07,33.0,23.0,Sunny intervals and a gentle breeze
7,22-11-08,33.0,23.0,Sunny intervals and a gentle breeze
8,22-11-09,34.0,24.0,Sunny intervals and a gentle breeze
9,22-11-10,34.0,24.0,Sunny intervals and a gentle breeze


Extract the name of the city for which data is gathered.

In [ ]:
#location = soup.find('div', attrs={'class':'wr-c-location'})
location = soup.find('h1', attrs={'id':'wr-location-name-id'})
location.text.split()

['Mumbai', '-', 'Weather', 'warnings', 'issued']

In [ ]:
# create a recording
filename_csv = location.text.split()[0]+'.csv'
df.to_csv(filename_csv, index=None)

In [ ]:
filename_xlsx = location.text.split()[0]+'.xlsx'
df.to_excel(filename_xlsx)